## 1. Get the name, personal id and kick-off date of campagin from C-span

In [1]:
import requests
import pandas as pd
import re
import calendar


from bs4 import BeautifulSoup

In [2]:
url = "https://www.c-span.org/series/?roadToTheWhiteHouse#candidates-tab"
response = requests.get(url)
doc = BeautifulSoup(response.text)

In [3]:
rows = []
candidates = doc.find_all(class_="D")
for candidate in candidates:
    row ={}
    try:
        row['name'] = candidate.find(class_="candidate D").text
    except:
        pass
    try:
        row['kickoff_date'] = candidate.find(class_="datetime").text
    except:
        pass
    try:
        row['person_id'] = candidate.find(class_="search-candidate")['href']
    except:
        pass
    rows.append(row)

In [4]:
df = pd.DataFrame(rows)
df = df.dropna(subset=['name'])
df = df[['name', 'kickoff_date', 'person_id']]

In [5]:
df['kickoff_date'] = df['kickoff_date'].astype(str)

In [6]:
df['kickoff_date'] = df['kickoff_date'].str.extract(r":(.*)")

In [7]:
df['person_id'] = df['person_id'].str.extract(r"=(\d+)$")
df

,name,kickoff_date,person_id
0,Michael Bennet,"May 2, 2019",1031622
2,Joe Biden Jr.,"April 25, 2019",34
4,Cory Booker,"February 1, 2019",84679
6,Steve Bullock,"May 14, 2019",9276486
8,Pete Buttigieg,"April 14, 2019",106618
10,Julian Castro,"January 12, 2019",9278200
12,John K. Delaney,"August 10, 2017",67856
14,Tulsi Gabbard,"January 11, 2019",1025291
16,Kirsten E. Gillibrand,"January 14, 2019",1022862
18,Kamala D. Harris,"January 21, 2019",1018696


In [8]:
df['kickoff_date'] = pd.to_datetime(df['kickoff_date'])

In [9]:
df

,name,kickoff_date,person_id
0,Michael Bennet,2019-05-02,1031622
2,Joe Biden Jr.,2019-04-25,34
4,Cory Booker,2019-02-01,84679
6,Steve Bullock,2019-05-14,9276486
8,Pete Buttigieg,2019-04-14,106618
10,Julian Castro,2019-01-12,9278200
12,John K. Delaney,2017-08-10,67856
14,Tulsi Gabbard,2019-01-11,1025291
16,Kirsten E. Gillibrand,2019-01-14,1022862
18,Kamala D. Harris,2019-01-21,1018696


In [10]:
df.to_csv("candidates_info.csv", index=False)

## 2. Get speech and event stops of Bernie Sanders
### 2.1. Scrape Bernie Sanders' speech coverage at C-span

In [11]:
url = "https://www.c-span.org/search/?empty-date=1&sdate=&edate=&searchtype=Videos&sort=Most+Recent+Airing&text=0&&personid%5B%5D=994&formatid%5B%5D=55&show100="
response = requests.get(url)
doc = BeautifulSoup(response.text)  

In [12]:
rows = []
videos = doc.find(class_="video-results").find_all('li')
for video in videos:
    row={}
    row['date'] = video.find(class_="time").text
    row['place'] = video.find('h3').text
    row['summary'] = video.find(class_="abstract").text
    row['url'] = video.find('a')['href']
    rows.append(row)

In [13]:
df_sanders = pd.DataFrame(rows)
df_sanders.head()

,date,place,summary,url
0,"June 23, 2019",Senator Bernie Sanders Town Hall at Clinton Co...,"Senator Bernie Sanders (I-VT), a 2020 Democrat...",//www.c-span.org/video/?461881-1/senator-berni...
1,"June 22, 2019",South Carolina Democratic Party Convention: Se...,"Senator Bernie Sanders (I-VT), a 2020 Democrat...",//www.c-span.org/video/?461983-8/south-carolin...
2,"June 12, 2019",Senator Bernie Sanders Remarks on Democratic S...,"Senator Bernie Sanders (I-VT), a 2020 Democrat...",//www.c-span.org/video/?461581-1/senator-berni...
3,"June 9, 2019",Iowa Democratic Party Hall of Fame Forum,The Iowa Democratic Party held its Hall of Fam...,//www.c-span.org/video/?461356-1/white-house-h...
4,"May 25, 2019","Senator Bernie Sanders Rally in Montpelier, Ve...","Senator Bernie Sanders (I-VT), a 2020 Democrat...",//www.c-span.org/video/?460966-1/senator-berni...


In [14]:
df_sanders['date'] = pd.to_datetime(df_sanders['date'])
df_sanders['summary'] = df_sanders.summary.astype(str)
df_sanders = df_sanders[df_sanders.summary.str.contains('Sanders')]
df_sanders

,date,place,summary,url
0,2019-06-23,Senator Bernie Sanders Town Hall at Clinton Co...,"Senator Bernie Sanders (I-VT), a 2020 Democrat...",//www.c-span.org/video/?461881-1/senator-berni...
1,2019-06-22,South Carolina Democratic Party Convention: Se...,"Senator Bernie Sanders (I-VT), a 2020 Democrat...",//www.c-span.org/video/?461983-8/south-carolin...
2,2019-06-12,Senator Bernie Sanders Remarks on Democratic S...,"Senator Bernie Sanders (I-VT), a 2020 Democrat...",//www.c-span.org/video/?461581-1/senator-berni...
4,2019-05-25,"Senator Bernie Sanders Rally in Montpelier, Ve...","Senator Bernie Sanders (I-VT), a 2020 Democrat...",//www.c-span.org/video/?460966-1/senator-berni...
5,2019-05-04,Senator Bernie Sanders in Iowa,"Senator Bernie Sanders (I-VT), a 2020 Democrat...",//www.c-span.org/video/?460388-1/senator-berni...
6,2019-04-12,"Senator Sanders Campaign Rally in Madison, Wis...","Senator Bernie Sanders (I-VT), a 2020 Democrat...",//www.c-span.org/video/?459755-1/senator-berni...
7,2019-03-10,Senator Bernie Sanders Campaign Event in Conco...,Senator Bernie Sanders (I-VT) hosted a rally i...,//www.c-span.org/video/?458676-1/senator-berni...
8,2019-03-02,Senator Bernie Sanders Presidential Campaign A...,Senator Bernie Sanders (I-VT) formally announc...,//www.c-span.org/video/?458403-1/senator-berni...
9,2019-02-19,Senator Bernie Sanders Presidential Campaign A...,"Senator Bernie Sanders (I-VT), a 2016 Democrat...",//www.c-span.org/video/?458032-1/senator-berni...
11,2018-11-27,Where We Go From Here,Senator Bernie Sanders (I-VT) laid out his pla...,//www.c-span.org/video/?454807-1/senator-berni...


In [15]:
sanders_kickoff = df[df.name == "Bernie Sanders"]['kickoff_date']
sanders_kickoff

32   2019-02-19
Name: kickoff_date, dtype: datetime64[ns]

In [16]:
df_sanders = df_sanders[df_sanders['date'] > '2019-02-18']

In [17]:
df_sanders['url'] = df_sanders['url'].str.replace("//www.c-span.org/video", "")
df_sanders['place'] = df_sanders['place'].str.replace("Senator Bernie Sanders", "")
df_sanders

/Users/leiao/.pyenv/versions/3.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/leiao/.pyenv/versions/3.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,date,place,summary,url
0,2019-06-23,Town Hall at Clinton College,"Senator Bernie Sanders (I-VT), a 2020 Democrat...",/?461881-1/senator-bernie-sanders-town-hall-cl...
1,2019-06-22,South Carolina Democratic Party Convention:,"Senator Bernie Sanders (I-VT), a 2020 Democrat...",/?461983-8/south-carolina-democratic-party-con...
2,2019-06-12,Remarks on Democratic Socialism,"Senator Bernie Sanders (I-VT), a 2020 Democrat...",/?461581-1/senator-bernie-sanders-delivers-rem...
4,2019-05-25,"Rally in Montpelier, Vermont","Senator Bernie Sanders (I-VT), a 2020 Democrat...",/?460966-1/senator-bernie-sanders-rally-montpe...
5,2019-05-04,in Iowa,"Senator Bernie Sanders (I-VT), a 2020 Democrat...",/?460388-1/senator-bernie-sanders-campaigns-am...
6,2019-04-12,"Senator Sanders Campaign Rally in Madison, Wis...","Senator Bernie Sanders (I-VT), a 2020 Democrat...",/?459755-1/senator-bernie-sanders-campaigns-ma...
7,2019-03-10,"Campaign Event in Concord, New Hampshire",Senator Bernie Sanders (I-VT) hosted a rally i...,/?458676-1/senator-bernie-sanders-campaigns-ha...
8,2019-03-02,Presidential Campaign Announcement,Senator Bernie Sanders (I-VT) formally announc...,/?458403-1/senator-bernie-sanders-launches-pre...
9,2019-02-19,Presidential Campaign Announcement,"Senator Bernie Sanders (I-VT), a 2016 Democrat...",/?458032-1/senator-bernie-sanders-announces-20...


In [18]:
df_sanders.to_csv("sanders_info.csv", index = False)

### 2.2. Scrape closed captioning transcripts for each speech of Bernie Sanders after his campagin started

In [19]:
def scrape_page(row):
    url_sanders = f"https://www.c-span.org/video{row['url']}"
    
    from selenium import webdriver
    driver = webdriver.Chrome()
    driver.get(url_sanders)
    
    buttons = driver.find_elements_by_class_name("link-show-full-text")
    for button in buttons:
        button.click()
    
    page = {}
    page['speech'] = '\n'.join([transcript.text.lower() for transcript in driver.find_elements_by_class_name("short_transcript")])
    
    return pd.Series(page)

In [20]:
scrape_page(df_sanders.loc[2])

speech    thank you for being here. let me say, my frien...
dtype: object

In [21]:
scraped_df = df_sanders.apply(scrape_page,axis=1)

In [22]:
scraped_df.to_csv("sanders_speech.csv", index=False)

In [23]:
df_sanders = df_sanders.merge(scraped_df, left_index=True, right_index=True)
df_sanders

,date,place,summary,url,speech
0,2019-06-23,Town Hall at Clinton College,"Senator Bernie Sanders (I-VT), a 2020 Democrat...",/?461881-1/senator-bernie-sanders-town-hall-cl...,of the white house? [cheers and applause] well...
1,2019-06-22,South Carolina Democratic Party Convention:,"Senator Bernie Sanders (I-VT), a 2020 Democrat...",/?461983-8/south-carolina-democratic-party-con...,hello south carolina democrats. my name is kir...
2,2019-06-12,Remarks on Democratic Socialism,"Senator Bernie Sanders (I-VT), a 2020 Democrat...",/?461581-1/senator-bernie-sanders-delivers-rem...,"thank you for being here. let me say, my frien..."
4,2019-05-25,"Rally in Montpelier, Vermont","Senator Bernie Sanders (I-VT), a 2020 Democrat...",/?460966-1/senator-bernie-sanders-rally-montpe...,"hello, everybody! it is a great honor to be he..."
5,2019-05-04,in Iowa,"Senator Bernie Sanders (I-VT), a 2020 Democrat...",/?460388-1/senator-bernie-sanders-campaigns-am...,i hate to let you down. this is not bernie. [c...
6,2019-04-12,"Senator Sanders Campaign Rally in Madison, Wis...","Senator Bernie Sanders (I-VT), a 2020 Democrat...",/?459755-1/senator-bernie-sanders-campaigns-ma...,"up next, senator bernie sanders hold as rally ..."
7,2019-03-10,"Campaign Event in Concord, New Hampshire",Senator Bernie Sanders (I-VT) hosted a rally i...,/?458676-1/senator-bernie-sanders-campaigns-ha...,thank you all. thank you. i know we new englan...
8,2019-03-02,Presidential Campaign Announcement,Senator Bernie Sanders (I-VT) formally announc...,/?458403-1/senator-bernie-sanders-launches-pre...,let me thank you all for a wonderful rendition...
9,2019-02-19,Presidential Campaign Announcement,"Senator Bernie Sanders (I-VT), a 2016 Democrat...",/?458032-1/senator-bernie-sanders-announces-20...,senator bernie sanders released a video announ...


In [24]:
df_sanders.to_csv("sanders_info_speech.csv", index=False)

### 2.3. Copy from Sanders' facebook event page and use regrex in visual studio to get sanders_events.txt

In [25]:
colnames=['date', 'city'] 

In [26]:
df_sanders_events = pd.read_csv("sanders_events.txt", sep=',',names=colnames)
df_sanders_events['date'] = pd.to_datetime(df_sanders_events['date'])
df_sanders_events 

,date,city
0,2019-06-23,"Rock Hill, SC"
1,2019-06-22,"Columbia, SC"
2,2019-06-20,"Laconia, NH"
3,2019-06-20,"Portsmouth, NH"
4,2019-06-20,"Keene, NH"
5,2019-06-20,"Nashua, NH"
6,2019-06-20,"Concord, NH"
7,2019-06-20,"Hooksett, NH"
8,2019-06-20,"Dover, NH"
9,2019-06-16,"Iowa City, IA"


### 2.4. Merge Sanders' event info and speech info together 

In [27]:
df_sanders = df_sanders_events.merge(df_sanders, on='date', how='left')

In [28]:
df_sanders

,date,city,place,summary,url,speech
0,2019-06-23,"Rock Hill, SC",Town Hall at Clinton College,"Senator Bernie Sanders (I-VT), a 2020 Democrat...",/?461881-1/senator-bernie-sanders-town-hall-cl...,of the white house? [cheers and applause] well...
1,2019-06-22,"Columbia, SC",South Carolina Democratic Party Convention:,"Senator Bernie Sanders (I-VT), a 2020 Democrat...",/?461983-8/south-carolina-democratic-party-con...,hello south carolina democrats. my name is kir...
2,2019-06-20,"Laconia, NH",NaN,NaN,NaN,NaN
3,2019-06-20,"Portsmouth, NH",NaN,NaN,NaN,NaN
4,2019-06-20,"Keene, NH",NaN,NaN,NaN,NaN
5,2019-06-20,"Nashua, NH",NaN,NaN,NaN,NaN
6,2019-06-20,"Concord, NH",NaN,NaN,NaN,NaN
7,2019-06-20,"Hooksett, NH",NaN,NaN,NaN,NaN
8,2019-06-20,"Dover, NH",NaN,NaN,NaN,NaN
9,2019-06-16,"Iowa City, IA",NaN,NaN,NaN,NaN


In [29]:
del df_sanders['place']

In [30]:
df_sanders.head()

,date,city,summary,url,speech
0,2019-06-23,"Rock Hill, SC","Senator Bernie Sanders (I-VT), a 2020 Democrat...",/?461881-1/senator-bernie-sanders-town-hall-cl...,of the white house? [cheers and applause] well...
1,2019-06-22,"Columbia, SC","Senator Bernie Sanders (I-VT), a 2020 Democrat...",/?461983-8/south-carolina-democratic-party-con...,hello south carolina democrats. my name is kir...
2,2019-06-20,"Laconia, NH",NaN,NaN,NaN
3,2019-06-20,"Portsmouth, NH",NaN,NaN,NaN
4,2019-06-20,"Keene, NH",NaN,NaN,NaN


In [31]:
df_sanders.to_csv("sanders.csv", index=False)

## 3. Repeat the above procedures for another candicate Kamala Harris

In [32]:
url = "https://www.c-span.org/search/?empty-date=1&sdate=&edate=&searchtype=Videos&sort=Most+Recent+Airing&text=0&&personid%5B%5D=1018696&formatid%5B%5D=55&show100="
response = requests.get(url)
doc = BeautifulSoup(response.text)

In [33]:
rows = []
videos = doc.find(class_="video-results").find_all('li')
for video in videos:
    row={}
    try:
        row['date'] = video.find(class_="time").text
    except:
        pass
    try:
        row['place'] = video.find('h3').text
    except:
        pass
    try:
        row['summary'] = video.find(class_="abstract").text
    except:
        pass
    row['url'] = video.find('a')['href']
    rows.append(row)

In [34]:
df_harris = pd.DataFrame(rows)
df_harris['date'] = pd.to_datetime(df_harris['date'])

In [35]:
df[df['name'] == "Kamala D. Harris"]['kickoff_date']

18   2019-01-21
Name: kickoff_date, dtype: datetime64[ns]

In [36]:
df_harris = df_harris[df_harris['date'] > '2019-01-20']

In [37]:
df_harris

,date,place,summary,url
0,2019-07-03,"Senator Kamala Harris Town Hall in Des Moines,...","Senator Kamala Harris (D-CA), a 2020 Democrati...",//www.c-span.org/video/?462218-1/senator-kamal...
1,2019-06-22,South Carolina Democratic Party Convention: Se...,"Senator Kamala Harris (D-CA), a 2020 Democrati...",//www.c-span.org/video/?461983-2/south-carolin...
2,2019-06-09,Iowa Democratic Party Hall of Fame Forum,The Iowa Democratic Party held its Hall of Fam...,//www.c-span.org/video/?461356-1/white-house-h...
3,2019-04-23,"Senator Kamala Harris Town Hall in Hanover, Ne...",Democratic presidential candidate and U.S. Sen...,//www.c-span.org/video/?459979-1/senator-kamal...
5,2019-02-19,Politics and Eggs Breakfast with Senator Kamal...,"Senator Kamala Harris (D-CA), a 2020 president...",//www.c-span.org/video/?457898-1/senator-kamal...
6,2019-02-18,"Senator Kamala Harris Town Hall in Portsmouth,...",Senator Kamala Harris (D-CA) held a town hall ...,//www.c-span.org/video/?457973-1/senator-kamal...


In [38]:
df_harris.to_csv("harris_info.csv", index=False)

In [39]:
def scrape_page(row):
    url_harris = f"https:{row['url']}"
    
    from selenium import webdriver
    driver = webdriver.Chrome()
    driver.get(url_harris)
    
    buttons = driver.find_elements_by_class_name("link-show-full-text")
    for button in buttons:
        button.click()
    
    page = {}
    page['speech'] = '\n'.join([transcript.text.lower() for transcript in driver.find_elements_by_class_name("short_transcript")])
    
    return pd.Series(page)

In [40]:
scrape_page(df_harris.loc[0])

speech    the first african-american and first woman to ...
dtype: object

In [41]:
scraped_df = df_harris.apply(scrape_page,axis=1)

In [42]:
df_harris = df_harris.merge(scraped_df, left_index=True, right_index=True)

In [43]:
df_harris

,date,place,summary,url,speech
0,2019-07-03,"Senator Kamala Harris Town Hall in Des Moines,...","Senator Kamala Harris (D-CA), a 2020 Democrati...",//www.c-span.org/video/?462218-1/senator-kamal...,the first african-american and first woman to ...
1,2019-06-22,South Carolina Democratic Party Convention: Se...,"Senator Kamala Harris (D-CA), a 2020 Democrati...",//www.c-span.org/video/?461983-2/south-carolin...,"what's up, south carolina? it's good to be in ..."
2,2019-06-09,Iowa Democratic Party Hall of Fame Forum,The Iowa Democratic Party held its Hall of Fam...,//www.c-span.org/video/?461356-1/white-house-h...,
3,2019-04-23,"Senator Kamala Harris Town Hall in Hanover, Ne...",Democratic presidential candidate and U.S. Sen...,//www.c-span.org/video/?459979-1/senator-kamal...,senator kamala harris officially launched her ...
5,2019-02-19,Politics and Eggs Breakfast with Senator Kamal...,"Senator Kamala Harris (D-CA), a 2020 president...",//www.c-span.org/video/?457898-1/senator-kamal...,and the people will hear all of it. [cheers an...
6,2019-02-18,"Senator Kamala Harris Town Hall in Portsmouth,...",Senator Kamala Harris (D-CA) held a town hall ...,//www.c-span.org/video/?457973-1/senator-kamal...,our mission statement states that we celebrate...


In [44]:
df_harris.to_csv("harris_info_speech.csv", index=False)

In [45]:
colnames=['date', 'city'] 

In [46]:
df_harris_events = pd.read_csv("harris_events", sep=',',names=colnames)
df_harris_events['date'] = pd.to_datetime(df_harris_events['date'])
df_harris_events 

,date,city
0,2019-06-15,"Las Vegas, NV"
1,2019-06-13,"Columbia, SC"
2,2019-06-10,"Dubuque, IA"
3,2019-06-09,"Waterloo, IA"
4,2019-05-29,"Greenville, SC"
5,2019-05-29,"Anderson, SC"
6,2019-05-19,"California City, CA"
7,2019-05-15,"Nashua, NH"
8,2019-04-23,"Hanover, NH"
9,2019-04-23,"Keene, NH"


In [47]:
df_harris = df_harris_events.merge(df_harris, on='date', how='left')

In [48]:
del df_harris['place']

In [49]:
df_harris

,date,city,summary,url,speech
0,2019-06-15,"Las Vegas, NV",NaN,NaN,NaN
1,2019-06-13,"Columbia, SC",NaN,NaN,NaN
2,2019-06-10,"Dubuque, IA",NaN,NaN,NaN
3,2019-06-09,"Waterloo, IA",The Iowa Democratic Party held its Hall of Fam...,//www.c-span.org/video/?461356-1/white-house-h...,
4,2019-05-29,"Greenville, SC",NaN,NaN,NaN
5,2019-05-29,"Anderson, SC",NaN,NaN,NaN
6,2019-05-19,"California City, CA",NaN,NaN,NaN
7,2019-05-15,"Nashua, NH",NaN,NaN,NaN
8,2019-04-23,"Hanover, NH",Democratic presidential candidate and U.S. Sen...,//www.c-span.org/video/?459979-1/senator-kamal...,senator kamala harris officially launched her ...
9,2019-04-23,"Keene, NH",Democratic presidential candidate and U.S. Sen...,//www.c-span.org/video/?459979-1/senator-kamal...,senator kamala harris officially launched her ...


In [50]:
df_sanders.to_csv("harris.csv", index=False)

## 4. Repeat the above procedures for another candicate Andrew Yang, except that Yang's event schedule is provided by his campagin team, which was cleaned through visual studio too

In [51]:
url = "https://www.c-span.org/search/?empty-date=1&sdate=&edate=&searchtype=Videos&sort=Most+Recent+Airing&text=0&&personid%5B%5D=96807&formatid%5B%5D=55&show100="

response = requests.get(url)
doc = BeautifulSoup(response.text) 

In [52]:
rows = []
videos = doc.find(class_="video-results").find_all('li')
for video in videos:
    row={}
    try:
        row['date'] = video.find(class_="time").text
    except:
        pass
    try:
        row['place'] = video.find('h3').text
    except:
        pass
    try:
        row['summary'] = video.find(class_="abstract").text
    except:
        pass
    row['url'] = video.find('a')['href']
    rows.append(row)

In [53]:
df_yang = pd.DataFrame(rows)
df_yang = df_yang[0:4]
df_yang['date'] = pd.to_datetime(df_yang['date'])
df_yang

,date,place,summary,url
0,2019-06-22,South Carolina Democratic Party Convention: Se...,"Senator Kirsten Gillibrand (D-NY) , Andrew Yan...",//www.c-span.org/video/?461983-9/south-carolin...
1,2019-06-09,Iowa Democratic Party Hall of Fame Forum,The Iowa Democratic Party held its Hall of Fam...,//www.c-span.org/video/?461356-1/white-house-h...
2,2019-04-15,Andrew Yang Rally at Lincoln Memorial,"Andrew Yang, a 2020 Democratic presidential ca...",//www.c-span.org/video/?459819-1/democratic-pr...
3,2019-03-20,"Andrew Yang in Plymouth, New Hampshire","Andrew Yang, a technology entrepreneur and 202...",//www.c-span.org/video/?458963-1/andrew-yang-m...


In [54]:
df_yang.to_csv("yang_info.csv", index=False)

In [55]:
def scrape_page(row):
    url_yang = f"https:{row['url']}"
    
    from selenium import webdriver
    driver = webdriver.Chrome()
    driver.get(url_yang)
    
    buttons = driver.find_elements_by_class_name("link-show-full-text")
    for button in buttons:
        button.click()
    
    page = {}
    page['speech'] = '\n'.join([transcript.text.lower() for transcript in driver.find_elements_by_class_name("short_transcript")])
    
    return pd.Series(page)

In [56]:
scrape_page(df_yang.loc[1])

speech    i have a lot of challenges with falling asleep...
dtype: object

In [57]:
scraped_df = df_yang.apply(scrape_page,axis=1)
scraped_df

,speech
0,greetings. greetings. my name is jay inslee. i...
1,i have a lot of challenges with falling asleep...
2,let's see what our viewers are thinking about....
3,andrew yang is running for the democratic nomi...


In [58]:
df_yang = df_yang.merge(scraped_df, left_index=True, right_index=True)
df_yang

,date,place,summary,url,speech
0,2019-06-22,South Carolina Democratic Party Convention: Se...,"Senator Kirsten Gillibrand (D-NY) , Andrew Yan...",//www.c-span.org/video/?461983-9/south-carolin...,greetings. greetings. my name is jay inslee. i...
1,2019-06-09,Iowa Democratic Party Hall of Fame Forum,The Iowa Democratic Party held its Hall of Fam...,//www.c-span.org/video/?461356-1/white-house-h...,i have a lot of challenges with falling asleep...
2,2019-04-15,Andrew Yang Rally at Lincoln Memorial,"Andrew Yang, a 2020 Democratic presidential ca...",//www.c-span.org/video/?459819-1/democratic-pr...,let's see what our viewers are thinking about....
3,2019-03-20,"Andrew Yang in Plymouth, New Hampshire","Andrew Yang, a technology entrepreneur and 202...",//www.c-span.org/video/?458963-1/andrew-yang-m...,andrew yang is running for the democratic nomi...


In [59]:
df_yang.to_csv("yang_info_speech.csv", index=False)

In [60]:
colnames=['date', 'city'] 

In [61]:
df_yang_events = pd.read_csv("yang_events", sep=',',names=colnames)

In [62]:
df_yang_events['date'] = pd.to_datetime(df_yang_events['date'])

In [63]:
df_yang_events

,date,city
0,2019-06-14,"Dover, NH"
1,2019-06-13,"West Ossipee, NH"
2,2019-05-18,"Columbia, SC"
3,2019-05-11,"Windham, NH"
4,2019-05-14,"New York, NY"
5,2019-05-11,"Milford, NH"
6,2019-05-11,"Claremont, NH"
7,2019-05-10,"Portsmouth, NH"
8,2019-05-09,"Hanover, NH"
9,2019-05-09,"Lebanon, NH"


In [64]:
df_yang = df_yang_events.merge(df_yang, on='date', how='left')

In [65]:
del df_yang['place']

In [66]:
df_yang

,date,city,summary,url,speech
0,2019-06-14,"Dover, NH",NaN,NaN,NaN
1,2019-06-13,"West Ossipee, NH",NaN,NaN,NaN
2,2019-05-18,"Columbia, SC",NaN,NaN,NaN
3,2019-05-11,"Windham, NH",NaN,NaN,NaN
4,2019-05-14,"New York, NY",NaN,NaN,NaN
5,2019-05-11,"Milford, NH",NaN,NaN,NaN
6,2019-05-11,"Claremont, NH",NaN,NaN,NaN
7,2019-05-10,"Portsmouth, NH",NaN,NaN,NaN
8,2019-05-09,"Hanover, NH",NaN,NaN,NaN
9,2019-05-09,"Lebanon, NH",NaN,NaN,NaN


In [67]:
df_yang.to_csv("yang.csv", index=False)

## 5. Using Google maps API to get geometries of cities

### 5.1. Get the API key of google map

In [68]:
import os
import time

In [69]:
from dotenv import load_dotenv
load_dotenv()

True

In [70]:
!pip install python-dotenv

sqlalchemy-migrate 0.12.0 has requirement SQLAlchemy>=0.9.6, but you'll have sqlalchemy 0.7.10 which is incompatible.
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [71]:
!touch .env

In [72]:
API_KEY = os.getenv("GMAP_API_KEY")

### 5.2. Combine the data of three candidates into one dataframe, giving them each an id, and get all the cities of the dataframe

In [73]:
df_sanders['id'] = 1
df_sanders['candidate'] = 'Sanders'
df_sanders['color'] = "#FF2311"

df_harris['id'] = 2
df_harris['candidate'] = 'Harris'
df_harris['color'] = "#009800"

df_yang['id'] = 3
df_yang['candidate'] = 'Yang'
df_yang['color'] = "#470AF5"

In [74]:
df = df_sanders.append(df_harris)
df = df.append(df_yang)
df.to_csv('candidates.csv',index=False)

In [75]:
placenames = []
for city in df.city:
    placenames.append(city)
placenames    

['Rock Hill, SC',
 'Columbia, SC',
 'Laconia, NH',
 'Portsmouth, NH',
 'Keene, NH',
 'Nashua, NH',
 'Concord, NH',
 'Hooksett, NH',
 'Dover, NH',
 'Iowa City, IA',
 'Rapids, IA',
 'Ottumwa, IA',
 'Waterloo, IA',
 'Bluffs, IA',
 'Ames, IA',
 'Des Moines, IA',
 'Dubuque, IA',
 'Waterloo, IA',
 'Marchshalltown, IA',
 'Des Moines, IA ',
 'San Jose, CA ',
 'Pasadena, CA ',
 'Las Vegas, NV',
 'Reno, NV ',
 'Manchester, NH',
 'Londonderry, NH',
 'Concord, NH',
 'Rollinsford, NH ',
 'Laconia, NH ',
 'Warner, NH',
 'Montpelier, VT',
 'Montgomery, AL',
 'Birmingham, AL',
 'Augusta, GA',
 'DenMarchk, SC',
 'Orangeburg, SC',
 'Charlotte, NC ',
 'Asheville, NC',
 'Sioux City, IA',
 'Spencer, IA ',
 'Fort Dodge',
 'Bldg Perry, IA',
 'Ames, IA',
 'Worth, TX',
 'DHouston, TX',
 'Greenville, SC',
 'Pittsburgh, PA',
 'Warren, MI ',
 'Madison, WI',
 'Malcom, IA',
 'Oskaloosa, IA',
 'Fairfield, Iowa',
 'Burlington, IA',
 'Muscatine',
 'Davenport, IA',
 'San Francisco, CA',
 'Los Angeles, CA',
 'San Diego,

### 5.3. Get geometries of cities

In [76]:
geometry_df = []
for place in placenames:
    url = 'https://maps.googleapis.com/maps/api/geocode/json'
    keys = {'address': place, 'key': API_KEY}
    r = requests.get(url,params=keys)
    result_dic = r.json()
    lat = result_dic['results'][0]['geometry']['location']['lat']
    long = result_dic['results'][0]['geometry']['location']['lng']
    coords = []
    coords.append(long)
    coords.append(lat)
    mygeometry = {'place': place, 'geometry.type': 'Point','geometry.coordinates':coords}
    geometry_df.append(mygeometry)
    time.sleep(1)
geometry_df

[{'place': 'Rock Hill, SC',
  'geometry.type': 'Point',
  'geometry.coordinates': [-81.02507840000001, 34.9248667]},
 {'place': 'Columbia, SC',
  'geometry.type': 'Point',
  'geometry.coordinates': [-81.0348144, 34.0007104]},
 {'place': 'Laconia, NH',
  'geometry.type': 'Point',
  'geometry.coordinates': [-71.4703509, 43.5278546]},
 {'place': 'Portsmouth, NH',
  'geometry.type': 'Point',
  'geometry.coordinates': [-70.7625532, 43.0717552]},
 {'place': 'Keene, NH',
  'geometry.type': 'Point',
  'geometry.coordinates': [-72.2781409, 42.933692]},
 {'place': 'Nashua, NH',
  'geometry.type': 'Point',
  'geometry.coordinates': [-71.46756599999999, 42.7653662]},
 {'place': 'Concord, NH',
  'geometry.type': 'Point',
  'geometry.coordinates': [-71.5375718, 43.2081366]},
 {'place': 'Hooksett, NH',
  'geometry.type': 'Point',
  'geometry.coordinates': [-71.46512830000002, 43.0967213]},
 {'place': 'Dover, NH',
  'geometry.type': 'Point',
  'geometry.coordinates': [-70.8736698, 43.1978624]},
 {'pla

In [77]:
geometry_df = pd.DataFrame(geometry_df)

In [78]:
geometry_df

,geometry.coordinates,geometry.type,place
0,"[-81.02507840000001, 34.9248667]",Point,"Rock Hill, SC"
1,"[-81.0348144, 34.0007104]",Point,"Columbia, SC"
2,"[-71.4703509, 43.5278546]",Point,"Laconia, NH"
3,"[-70.7625532, 43.0717552]",Point,"Portsmouth, NH"
4,"[-72.2781409, 42.933692]",Point,"Keene, NH"
5,"[-71.46756599999999, 42.7653662]",Point,"Nashua, NH"
6,"[-71.5375718, 43.2081366]",Point,"Concord, NH"
7,"[-71.46512830000002, 43.0967213]",Point,"Hooksett, NH"
8,"[-70.8736698, 43.1978624]",Point,"Dover, NH"
9,"[-91.5301683, 41.6611277]",Point,"Iowa City, IA"


### 5.4. Merge the geometry data with the previous dataframe

In [79]:
df = df.merge(geometry_df, left_on='city',right_on='place')

In [80]:
df = df.sort_values(by="date", ascending=True)

In [81]:
df = df.drop_duplicates(subset=['date','candidate','city','url'], keep='first')

In [82]:
df = df.reset_index(drop=True)

In [83]:
df.rename(columns={'city': 'properties.name', 'speech': 'properties.article', 'date':'properties.headline', 'id':'properties.group_id','candidate':'properties.group_name', 'color':'properties.color'}, inplace=True)

In [84]:
df['properties.month']=df['properties.headline'].apply(lambda x: x.month)

In [85]:
df['properties.month']=df['properties.month'].apply(lambda x: calendar.month_name[x])

In [86]:
df = df[['geometry.coordinates', 'geometry.type', 'properties.headline','properties.month','properties.article','properties.group_name','properties.color','properties.group_id','properties.name']]

In [87]:
df['properties.headline'] = df['properties.headline'].astype(str)

In [88]:
df['properties.headline'] = df['properties.headline'] + ' || ' + df['properties.group_name'] + ' || ' + df['properties.name']

df['properties.article'] = df['properties.name'] + ' ' + df['properties.article'].map(str)

df['properties.article'] = df['properties.article'].str.replace('nan','')

In [89]:
df.dtypes
df.to_csv("geo-data.csv",index=False)
df

,geometry.coordinates,geometry.type,properties.headline,properties.month,properties.article,properties.group_name,properties.color,properties.group_id,properties.name
0,"[-122.2711639, 37.8043514]",Point,"2019-01-27 || Harris || Oakland, CA",January,"Oakland, CA",Harris,#009800,2,"Oakland, CA"
1,"[-93.6249593, 41.5868353]",Point,"2019-01-28 || Harris || Des Moines, IA",January,"Des Moines, IA",Harris,#009800,2,"Des Moines, IA"
2,"[-93.2650108, 44.977753]",Point,"2019-01-29 || Yang || Minneapolis, MN",January,"Minneapolis, MN",Yang,#470AF5,3,"Minneapolis, MN"
3,"[-92.91640500000001, 41.4080327]",Point,"2019-01-31 || Yang || Pella, IA",January,"Pella, IA",Yang,#470AF5,3,"Pella, IA"
4,"[-93.5574376, 41.3580484]",Point,"2019-01-31 || Yang || Indianola, IA",January,"Indianola, IA",Yang,#470AF5,3,"Indianola, IA"
5,"[-94.86727929999999, 42.0655986]",Point,"2019-02-01 || Yang || Carroll, IA",February,"Carroll, IA",Yang,#470AF5,3,"Carroll, IA"
6,"[-95.3588867, 42.024988]",Point,"2019-02-01 || Yang || Denison, IA",February,"Denison, IA",Yang,#470AF5,3,"Denison, IA"
7,"[-94.3774232, 42.0172457]",Point,"2019-02-01 || Yang || Jefferson, IA",February,"Jefferson, IA",Yang,#470AF5,3,"Jefferson, IA"
8,"[-72.2781409, 42.933692]",Point,"2019-02-05 || Yang || Keene, NH",February,"Keene, NH",Yang,#470AF5,3,"Keene, NH"
9,"[-71.1833857, 43.0362094]",Point,"2019-02-05 || Yang || Raymond, NH",February,"Raymond, NH",Yang,#470AF5,3,"Raymond, NH"


### 5.5. Change dataframe to geojason

In [90]:
!pip install json

  Could not find a version that satisfies the requirement json (from versions: )
No matching distribution found for json
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [91]:
import json

In [92]:
ok_json = json.loads(df.to_json(orient='records'))

In [93]:
def process_to_geojson(file):
    geo_data = {"type": "FeatureCollection", "features":[]}
    for row in file:
        this_dict = {"type": "Feature", "properties":{}, "geometry": {}}
        for key, value in row.items():
            key_names = key.split('.')
            if key_names[0] == 'geometry':
                this_dict['geometry'][key_names[1]] = value
            if str(key_names[0]) == 'properties':
                this_dict['properties'][key_names[1]] = value
        geo_data['features'].append(this_dict)
    return geo_data

In [94]:
geo_format = process_to_geojson(ok_json)

In [95]:
with open('geo-data.js', 'w') as outfile:
    outfile.write("var infoData = ")
with open('geo-data.js', 'a') as outfile:
    json.dump(geo_format, outfile)